# Capstone: Survey Existing Research and Reproduce Available Solutions 

# Related Papers
#### Source 1:
https://www.kaggle.com/datasets/akhatova/pcb-defects
https://www.researchgate.net/publication/332642034_TDD-Net_A_Tiny_Defect_Detection_Network_for_Printed_Circuit_Boards
https://github.com/Ixiaohuihuihui/Tiny-Defect-Detection-for-PCB
#### Source 2:
https://www.kaggle.com/datasets/kubeedgeianvs/pcb-aoi
https://ianvs.readthedocs.io/en/latest/proposals/test-reports/testing-single-task-learning-in-industrial-defect-detection-with-pcb-aoi.html
https://github.com/kubeedge/ianvs

In this execices, I will try to reproduce the result from [Source 1](https://www.kaggle.com/datasets/akhatova/pcb-defects) first, and then look into reproduce [Source 2](https://www.kaggle.com/datasets/kubeedgeianvs/pcb-aoi) as there is less reference code.

### Source 1 Reproduction
The PCB files can be downloaded here: https://www.kaggle.com/code/pinokiokr/pcb-defect-detection/input


In [ ]:
import pandas as pd
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
from tqdm import tqdm
import argparse
import glob
import xml.etree.ElementTree as ET 
import ast
import numpy as np
import cv2

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as immg

import random

import torch

import torchvision
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as T
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN

from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split


os.environ['CUDA_LAUNCH_BLOCKING'] = "1"


import warnings
warnings.filterwarnings("ignore")

# Get the current working directory
current_path = os.getcwd()

# Print the current path
print("Current Path:", current_path)

images_dir = current_path + '/PCB_DATASET/images'
annotations_dir = current_path + '/PCB_DATASET/Annotations'

# Count the number of images
# image_count = sum(len(files) for _, _, files in os.walk(images_dir))
# print(f"Number of images: {image_count}")
# annotated_image_count = sum(len(files) for _, _, files in os.walk(annotations_dir))
# print(f"Number of annotated images: {annotated_image_count}")

In [42]:
# !pip install albumentations==0.4.6
import albumentations as A
from albumentations.pytorch import ToTensorV2

### I. Generate CSV


In [ ]:
dataset = {
            "xmin":[],
            "ymin":[],   
            "xmax":[],
            "ymax":[],
            "class":[],    
            "file":[],
            "width":[],
            "height":[],
           }
all_files = []
# Files to exclude
excluded_files = {".DS_Store"}
for path, subdirs, files in os.walk(annotations_dir):
#     print([path, subdirs, files])
    filtered_files = [f for f in files if f not in excluded_files]
    for name in filtered_files:
        all_files.append(os.path.join(path, name))

# print(all_files)       
print(type(dataset))
print(dataset)

In [44]:
for anno in all_files:
    # print(anno)
    tree = ET.parse(anno)
    
    for elem in tree.iter():
        # print(elem)
        
        if 'size' in elem.tag:
            # print('[size] in elem.tag ==> list(elem)\n'), print(list(elem))
            for attr in list(elem):
                if 'width' in attr.tag: 
                    width = int(round(float(attr.text)))
                if 'height' in attr.tag:
                    height = int(round(float(attr.text)))    

        if 'object' in elem.tag:
            # print('[object] in elem.tag ==> list(elem)\n'), print(list(elem))
            for attr in list(elem):
                
                # print('attr = %s\n' % attr)
                if 'name' in attr.tag:
                    name = attr.text                 
                    dataset['class']+=[name]
                    dataset['width']+=[width]
                    dataset['height']+=[height] 
                    dataset['file']+=[anno.split('/')[-1][0:-4]] 
                            
                if 'bndbox' in attr.tag:
                    for dim in list(attr):
                        if 'xmin' in dim.tag:
                            xmin = int(round(float(dim.text)))
                            dataset['xmin']+=[xmin]
                        if 'ymin' in dim.tag:
                            ymin = int(round(float(dim.text)))
                            dataset['ymin']+=[ymin]                                
                        if 'xmax' in dim.tag:
                            xmax = int(round(float(dim.text)))
                            dataset['xmax']+=[xmax]                                
                        if 'ymax' in dim.tag:
                            ymax = int(round(float(dim.text)))
                            dataset['ymax']+=[ymax]    

In [ ]:
data=pd.DataFrame(dataset)
data

### II. Reading the CSV file

In [46]:
# partition the data into training and testing splits using 80% of
# the data for training and the remaining 20% for testing
train, test = train_test_split(data, shuffle=True, test_size=0.2, random_state=34)

In [ ]:
train.shape, test.shape

In [ ]:
train.head()

In [ ]:
test.head()

In [50]:
classes_la = {"missing_hole": 0, "mouse_bite": 1, "open_circuit":2, "short": 3, 'spur': 4,'spurious_copper':5}

train["class"] = train["class"].apply(lambda x: classes_la[x])
test["class"] = test["class"].apply(lambda x: classes_la[x])

In [ ]:
train.head()

In [ ]:
test.head()

### III. Visualization

In [ ]:
# PJC (deep copy)
df = train.copy()

df_grp = df.groupby(['file'])
print(df_grp)

In [ ]:
df_grp.size()

In [ ]:
# DataFrameGroupBy (https://steadiness-193.tistory.com/47)
image_name = '01_missing_hole_02'
image_group = df_grp.get_group(image_name)
print(image_group)

In [ ]:
bbox = image_group.loc[:,['xmin', 'ymin', 'xmax', 'ymax']]
print([bbox, type(bbox)])

In [57]:
def plot_image(image_name, images_dir):
    print(image_name)
    image_group = df_grp.get_group(image_name)
    bbox = image_group.loc[:,['xmin', 'ymin', 'xmax', 'ymax']]
    if "missing" in name.split('_'):
        images_dir += '/Missing_hole/'
    if "mouse" in name.split('_'):
        images_dir += '/Mouse_bite/'
    if "open" in name.split('_'):
        images_dir += '/Open_circuit/'
    if "short" in name.split('_'):
        images_dir += '/Short/'
    if "spur" in name.split('_'):
        images_dir += '/Spur/'
    if "spurious" in name.split('_'):
        images_dir += '/Spurious_copper/'
   
    img = immg.imread(images_dir+""+name+'.jpg')
    fig,ax = plt.subplots(figsize=(18,10))
    ax.imshow(img,cmap='binary')
    for i in range(len(bbox)):
        box = bbox.iloc[i].values
        print(box)
        x,y,w,h = box[0], box[1], box[2]-box[0], box[3]-box[1]
        rect = matplotlib.patches.Rectangle((x,y),w,h,linewidth=1,edgecolor='r',facecolor='none',)
        # ax.text(*box[:2], image_group["class"].values, verticalalignment='top', color='white', fontsize=13, weight='bold')
        ax.add_patch(rect)
    plt.show()

In [ ]:
name = '01_missing_hole_01'
plot_image(name, images_dir)

In [ ]:
name = train.file[500]
plot_image(name, images_dir)

In [ ]:
name = train.file[100]
plot_image(name, images_dir)

In [ ]:
name = train.file[105]
plot_image(name, images_dir)

### IV. Creating Custom database

In [62]:
class fcbData(object):
    def __init__(self, df, IMG_DIR, transforms): 
        self.df = df
        self.img_dir = IMG_DIR
        self.image_ids = self.df['file'].unique().tolist()
        self.transforms = transforms
        
    def __len__(self):
        return len(self.image_ids)
    
    def __getitem__(self, idx):
        image_id = self.image_ids[idx]
        a = ''
        if "missing" in image_id.split('_'):
            a = '/Missing_hole/'
        elif "mouse" in image_id.split('_'):
            a = '/Mouse_bite/'
        elif "open" in image_id.split('_'):
            a = '/Open_circuit/'
        elif "short" in image_id.split('_'):
            a = '/Short/'
        elif "spur" in image_id.split('_'):
            a = '/Spur/'
        elif "spurious" in image_id.split('_'):
            a = '/Spurious_copper/'
        image_values = self.df[self.df['file'] == image_id]
        image = cv2.imread(self.img_dir+a+image_id+".jpg",cv2.IMREAD_COLOR)
        print(self.img_dir+a+image_id+".jpg")
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image /= 255.0
        
        boxes = image_values[['xmin', 'ymin', 'xmax', 'ymax']].to_numpy()
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        
        labels = image_values["class"].values
        labels = torch.tensor(labels)
        
        target = {}
        target['boxes'] = boxes
        target['labels'] = labels
        target['image_id'] = torch.tensor([idx])
        target['area'] = torch.as_tensor(area, dtype=torch.float32)
        target['iscrowd'] = torch.zeros(len(classes_la), dtype=torch.int64)

        if self.transforms:
            sample = {
                'image': image,
                'bboxes': target['boxes'],
                'labels': labels
            }
        
            sample = self.transforms(**sample)
            image = sample['image']
            
            target['boxes'] = torch.stack(tuple(map(torch.tensor, zip(*sample['bboxes'])))).permute(1, 0)

        return torch.tensor(image), target, image_id

In [63]:
#pip install -U albumentations

In [64]:
def get_train_transform():
    return A.Compose([
        ToTensorV2(p=1.0)
    ], bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']})

def get_valid_transform():
    return A.Compose([
        ToTensorV2(p=1.0)
    ], bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']})

In [65]:
fcb_dataset   = fcbData(df, images_dir, get_train_transform())

In [ ]:
type(fcb_dataset[0]), len(fcb_dataset[0]), type(fcb_dataset[0][0]), type(fcb_dataset[0][1]), type(fcb_dataset[0][2])

In [ ]:
print([fcb_dataset[0][0], fcb_dataset[0][1], fcb_dataset[0][2]])

In [ ]:
#Check if the custom dataset object created ealier works
img, tar, _ = fcb_dataset[random.randint(0,50)]
bbox = tar['boxes']
fig,ax = plt.subplots(figsize=(18,10))
ax.imshow(img.permute(1,2,0).cpu().numpy())
for j in tar["labels"].tolist():
    classes_la = {0:"missing_hole", 1: "mouse_bite", 2:"open_circuit",3: "short", 4:'spur',5:'spurious_copper'}
    l = classes_la[j]
    for i in range(len(bbox)):
        box = bbox[i]
        x,y,w,h = box[0], box[1], box[2]-box[0], box[3]-box[1]
        rect = matplotlib.patches.Rectangle((x,y),w,h,linewidth=2,edgecolor='r',facecolor='none',)
        ax.text(*box[:2], l, verticalalignment='top', color='red', fontsize=13, weight='bold')
        ax.add_patch(rect)
    plt.show()

In [ ]:
len(df)

In [ ]:
#Split data into training and test
image_ids = df['file'].unique()
valid_ids = image_ids[-665:]
train_ids = image_ids[:-665]
valid_df = df[df['file'].isin(valid_ids)]
train_df = df[df['file'].isin(train_ids)]
train_df.shape,valid_df.shape

### V. Dataloader

In [ ]:
print(images_dir)

In [72]:
def collate_fn(batch):
    return tuple(zip(*batch))

In [73]:
train_dataset = fcbData(df, images_dir, get_train_transform())
valid_dataset = fcbData(df, images_dir, get_valid_transform())

# split the dataset in train and test set
indices = torch.randperm(len(train_dataset)).tolist()

train_data_loader = DataLoader(
    train_dataset,
    batch_size=1,
    shuffle=False,
    num_workers=0, #increase worker wont work inside jupytor notebook
    collate_fn=collate_fn
)

valid_data_loader = DataLoader(
    valid_dataset,
    batch_size=1,
    shuffle=False,
    num_workers=0,
    collate_fn=collate_fn
)

In [ ]:
next(iter(train_data_loader))

Most pretrained models are trained with a background class, we'll include it in our model, so in that case our number of classes will be 6

In [75]:
## num_classes = 6 # + background
num_classes = 6

# load a model; pre-trained on COCO
# .. fpn = 'feature pyramid network'
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

# get number of input features for the classifier
in_features = model.roi_heads.box_predictor.cls_score.in_features

# replace the pre-trained head with a new one
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

In [76]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [77]:
model.to(device)
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.Adam(params, lr=0.0001, weight_decay=0.0005,)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

In [78]:
num_epochs = 1

In [ ]:
train_data_loader

In [ ]:
import sys
best_epoch = 0
min_loss = sys.maxsize

for epoch in range(num_epochs):
    tk = tqdm(train_data_loader)
    model.train();
    for images, targets, image_ids in tk:
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        loss_dict = model(images, targets)

        losses = sum(loss for loss in loss_dict.values())
        loss_value = losses.item()

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()
        
        tk.set_postfix(train_loss=loss_value)
    tk.close()
    
    # update the learning rate
    if lr_scheduler is not None:
        lr_scheduler.step()
    
    print(f"Epoch #{epoch} loss: {loss_value}") 
        
    #validation 
    model.eval();
    with torch.no_grad():
        tk = tqdm(valid_data_loader)
        for images, targets, image_ids in tk:
            images = list(image.to(device) for image in images)
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
            val_output = model(images)
            val_output = [{k: v.to(device) for k, v in t.items()} for t in val_output]
            IOU = []
            for j in range(len(val_output)):
                a,b = val_output[j]['boxes'].cpu().detach(), targets[j]['boxes'].cpu().detach()
                chk = torchvision.ops.box_iou(a,b)
                res = np.nanmean(chk.sum(axis=1)/(chk>0).sum(axis=1))
                IOU.append(res)
            tk.set_postfix(IoU=np.mean(IOU))
        tk.close()

Sample evaluation on validation dataset image

In [ ]:
img,target,_ = valid_dataset[3]
# put the model in evaluation mode
model.eval()
with torch.no_grad():
    prediction = model([img.to(device)])[0]
    
print('predicted #boxes: ', len(prediction['boxes']))
print('real #boxes: ', len(target['boxes']))

In [42]:
torch.save(model.state_dict(), 'pcbdetection.pt')

### VII. Evaluation

In [ ]:
y_true = []
y_pred = []

for i in range(50):
    img, target, _ = valid_dataset[i]  # Load image and target
    model.eval()
    with torch.no_grad():
        # Get prediction
        prediction = model([img.to(device)])[0]

        # Ensure there is at least one label in prediction
        if len(prediction['labels']) > 0:
            y_true.append(target['labels'][0].item())  # Append the first label from target
            y_pred.append(prediction['labels'][0].item())  # Append the first predicted label
        else:
            print(f"No predictions for image {i}. Adding placeholder values.")
            y_true.append(target['labels'][0].item())  # Use the true label
            y_pred.append(-1)  # Placeholder for no prediction

In [ ]:
y_pred

In [45]:
yy_pred = []
for v in y_pred:
    if isinstance(v, torch.Tensor):  # Check if v is a tensor
        yy_pred.append(v.cpu().item())  # Move tensor to CPU and convert to Python scalar
    else:
        yy_pred.append(v)  # Append directly if v is already an integer

In [ ]:
yy_pred

In [ ]:
y_true

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_true, yy_pred)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_true, yy_pred))